# Initial data description

In [1]:
import pandas as pd
import matplotlib.pyplot as plt

Initial data ("Initial dataset" from Table II in [1])

In [2]:
df = pd.read_excel('initial_data_raw_CT.xlsx', index_col=0)
df.head()

,P (W),v (mm/s),h (mu m),dens
0,328,688,107,92.00
1,328,688,107,91.71
2,328,688,107,91.50
3,251,327,95,98.73
4,251,327,95,98.53


- **P**: laser power
- **v**: laser velocity 
- **h**: hatching distance

For each data point corresponding to a given set of process parameters and the measured density, we added two additional synthetic data points to ensure the stability of the GP algorithm. Here, we use a GP algorithm where the dispersion parameter is not a fixed hyperparameter but is instead fitted directly from the data. The inclusion of these synthetic points is also essential for accurately estimating the measurement uncertainty (see our [other paper](https://www.sciencedirect.com/science/article/pii/S0264127523001144)).

# Single-objective BO

Here we define Expected improvement (EI) directly.

In [3]:
import numpy as np
from scipy.stats import norm
import warnings

warnings.filterwarnings('ignore')


def lattice_for_prediction(bounds, step):
    """
    Generate a lattice with predefined bounds and dicretization rate (step) 
    """
        
    grid = np.meshgrid(*[np.linspace(bounds[i][0], bounds[i][1], step) for i in range(len(bounds))])
        
    X_lattice = np.concatenate([grid[i].reshape(-1, 1) for i in range(len(bounds))], axis=1)

    return X_lattice



def EI(mu, sigma, mu_sample, gpr, xi=0):
    '''
    Computes the EI at points X based on existing samples X_sample
    and Y_sample using a Gaussian process surrogate model.
    
    Returns:
        Expected improvements at points X.
    '''
    
    
    mu_sample_opt = np.min(mu_sample)

    with np.errstate(divide='warn'):
        imp = mu_sample_opt-mu - xi
        Z = imp / sigma
        ei = imp * norm.cdf(Z) + sigma * norm.pdf(Z)
        ei[sigma == 0.0] = 0.0
    return ei 


def propose_location(acquisition, mu, sigma, mu_sample, gpr, X_lattice):
    '''
    Proposes the next sampling point by optimizing the acquisition function by
    finding argmax EI(x)
    
    '''
    acq_for_lattice = acquisition(mu, sigma, mu_sample, gpr)
    
      
    return X_lattice[np.argmax(acq_for_lattice)]

In [4]:
from sklearn.preprocessing import MinMaxScaler 
from sklearn.metrics import mean_squared_error
from sklearn.gaussian_process.kernels import RBF, Matern, ConstantKernel as C

# Custom Gaussian Regression 
from scripts.gauss_custom import GaussianProcessRegressor 

coordinates = [ 'P (W)','v (mm/s)', 'h (mu m)']
target = 'dens' 

X = df[coordinates].astype(float)
target_vals = df[target].astype(float)

In [5]:
kernel = C(1.0, (1e-3, 1e3)) * Matern(length_scale=[3, 3, 3], length_scale_bounds=((1e-2, 1e5), (1e-2, 1e5), (1e-2, 1e5)),  nu=3/2)  
surr = GaussianProcessRegressor(kernel=kernel, alpha_fit=True, n_restarts_optimizer=55)

surr.fit(X, target_vals)

predicted = surr.predict(X)

print('RMSE, ', np.sqrt(mean_squared_error(target_vals, predicted )))

RMSE,  0.2269511012290325


We slightly extend the boundaries beyond the original data ranges by adding margins equal to 3% of each parameter's range. This ensures that the borders used for analysis or visualization are broader than the actual dataset, providing improved stability and clearer boundary 

In [6]:
bs0 = (df[coordinates[0]].max() - df[coordinates[0]].min())*3 /100 
bs1 = (df[coordinates[1]].max() - df[coordinates[1]].min())*3 /100 
bs2 =  (df[coordinates[2]].max() - df[coordinates[2]].min())*3 /100 


bounds = [
          [df[coordinates[0]].min() - bs0, df[coordinates[0]].max() +  bs0], 
          [df[coordinates[1]].min() - bs1, df[coordinates[1]].max()+  bs1],
          [df[coordinates[2]].min()- bs2, df[coordinates[2]].max()+  bs2]
          ]

X_lat = lattice_for_prediction(bounds, step=50)


mu, sigma = surr.predict(X_lat, return_std=True)
mu_sample = df.dens.values

In [7]:
proposed_X = propose_location(EI,  -mu, sigma, -mu_sample, surr, X_lat)

This is the point predicted by the algorithm (see "BO predicted (first iteration)" from [1]).

In [8]:
proposed_X

array([254.25816327, 120.47      , 108.71      ])